This Notebook launches a convergence study for the 1D Shock-Acoustic-Wave interaction problem as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r "BoSSSpad.dll"
#r "XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
Init();


In [16]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D_ConvergenceStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESTSF_ShockAcousticInteraction1D_ConvergenceStudy3'.
Default Execution queue is chosen for the database.
Creating database '\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy3'.


{ Session Count = 0; Grid Count = 0; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy3 }

In [3]:
BoSSSshell.GetDefaultQueue()

RuntimeLocation,win\amd64
AdditionalEnvironmentVars,"[ [OMP_PROC_BIND, spread] ]"
DeploymentBaseDirectory,\\dc3\userspace\sebastian\cluster\binaries
DeployRuntime,True
Name,Default
DotnetRuntime,dotnet
Username,FDY\sebastian
NumOfAdditionalServiceCores,0
NumOfAdditionalServiceCoresMPISerial,0
NumOfServiceCoresPerMPIprocess,0
ServerName,DC3


In [4]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy

In [5]:
int[] Configs = new int[] {1,2,3};
int[] DGdegrees = new int[] {3};
int[] Grid_Res = new int[] {0,1,3,7};
double[] Amplitudes = new double[] {1e-5};
string[] WaveForms= new string[] {"1sinus"};
string[] LevelSetConfigs = new string[] {"true"};//,"false"};
double[] MachLs = new double[] {1.5};
double[] WLs = new double[] {0.8};

In [17]:
//foreach(double iProb in iProbs) {
    double amp=1e-5;
    double amp_neg=0.0;
    double amp_pos=amp;
    double endTime=1.5;
    double shockPos=1.5;
    double wavePos=0.0;
    double scaling=1;
    double waveLength=0.4;
    double MachL=1.50;
                foreach(double waveLength in WLs ){
            foreach(double MachL in MachLs ){
        foreach(double amp in Amplitudes ){
foreach(double config in Configs) {
foreach(string waveform in WaveForms) {
    foreach(string withLS in LevelSetConfigs){
        amp_neg= 0.0;            amp_pos= amp;
         if(config==1){ //downstream negative acoustic wave (slow)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;          shockPos=0.5;            wavePos=0.9;
        }else if(config==2){ //upstream positiv acoustic wave (fast)
            amp_neg= 0.0;            amp_pos= amp;            endTime=1.5;            shockPos=1.5;            wavePos=0.0;
        }else if(config==3){ //upstream negative acoustic wave (still fast)
            amp_neg= amp;            amp_pos= 0.0;            endTime=4.0;           shockPos=1.5;            wavePos=0.3;
        }
    
    foreach(int p in DGdegrees) {
        foreach(int n in Grid_Res) {
            int cells=0;
            bool wLS;
            if(withLS == "true"){
                cells = (n+1)*15+1;wLS=true;
            }else{
                cells=(n+1)*15;wLS=false;
            }
                 var tmp_cntrl = XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength: waveLength ,scaling: scaling ,endTime: endTime ,MachL: MachL ,withLevelSet: wLS ,wavePosition: wavePos , shockPosition: shockPos ,p_amp_neg:  amp_neg , p_amp_pos: amp_pos ,waveform: waveform,dgDegree: p ,numOfCellsX: cells ,PlotInterval:-1,numOfCellsT: cells ,dbPath: dbPath );
                Job j = new Job(tmp_cntrl.SessionName, typeof(XESTSF.XESTSFMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
                j.RetryCount = 2;
                j.MySetCommandLineArguments("--control", $"cs:XESTSF.XESTSFHardCodedControl.AcousticWave1D(waveLength:{waveLength},scaling:{scaling},endTime:{endTime},MachL:{MachL},withLevelSet:{withLS},wavePosition:{wavePos}, shockPosition:{shockPos},p_amp_neg: {amp_neg}, p_amp_pos:{amp_pos},waveform:\"{waveform}\",dgDegree:{p},numOfCellsX:{cells},PlotInterval:-1,numOfCellsT:{cells},dbPath:@\"{dbPath}\")");
                j.NumberOfThreads=1;
                Console.WriteLine("!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!");
                j.Activate(BoSSSshell.GetDefaultQueue()); 
                
            
        };                                                                                                                                                          
    };
};
    
}
}
        }
            }
                }



!!!!!!!!!!!!!!!! WARNING: OPENMP Paralelization turned off !!!!!!!!!!!!
Deployments so far (0): ;
Success: 0
job submit count: 0


unable to determine job status - unknown


Deploying job AW_p3_xCells16_tCells16_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8 ... 
Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\sebastian\cluster\binaries\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy3-XESTSF2024May25_154235.830321
copied 53 files.
   copied 'win\amd64' runtime.
deployment finished.


Error: Command cancelled.

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(7*86400 , 60); // wait for one week, or until all jobs finish

## Check if residual norms converged

Load sessions

In [6]:
var sessions = BoSSSshell.WorkflowMgm.DefaultDatabase.Sessions;
sessions

#0: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/3/2024 10:53:36 AM	feb600d5...
#1: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/3/2024 10:54:58 AM	6e2ea10b...
#2: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/3/2024 10:56:21 AM	28ecdf1d...
#3: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/7/2024 12:50:01 PM	c30bcf0c...
#4: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/7/2024 12:49:34 PM	e430d224...
#5: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/7/2024 12:49:19 PM	0c1f4c55...
#6: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1

Obtain the residual norms

In [7]:
var ass= new MatrixAssembler(1);
var res = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).ResHistory.Last());
res

[ 4.17673720176366E-12, 5.745424648013963E-10, 3.0416885837344547E-10, 8.047201205560531E-11, 1.6398503260686175E-10, 1.5722681544382512E-12, 2.4581456062516657E-11, 2.279472651088386E-12, 1.9680356291241526E-12, 8.198354499437978E-13, 7.812169833683884E-13, 1.3755666637777938E-12, 9.511227708362726E-09, 3.7644845734528623E-07, 1.0555983868699228E-12, 5.328645130146802E-09, 3.127181863250669E-09, 5.741718232473919E-12 ]

In [8]:
var ass= new MatrixAssembler(1);
var enres = sessions.Select(si => ((IDTTimeStepInfo) ((TimestepProxy)si.Timesteps.Last()).GetInternal()).EnResHistory.Last());
enres

[ 2.0382282226127365E-05, 1.3645879809615809E-05, 1.6072711170265792E-05, 2.907085902784185E-06, 1.745809546847069E-05, 7.4543819520159555E-06, 1.557377954193541E-05, 2.305036964652607E-05, 2.5208634895048603E-05, 1.5485435579098796E-05, 5.367379964213245E-05, 2.7618519926095482E-05, 0.00017062160727410576, 3.8819234481294884E-05, 5.406976956619877E-05, 1.9282674140371255E-05, 1.71073686411359E-05, 1.765047249501271E-05 ]

Assert if smaller than threshold

In [15]:
res.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-06));
enres.ForEach(r => NUnit.Framework.Assert.IsTrue(r<1e-03));